# PhyloBayes estimation of amino-acids preferences 

We test the estimation of amino-acids preferences with PhyloBayes.
    1. Download newick tree and site-specific amino-acid preferences generated by deep mutational scanning in the lab 
    2. Use the data to run simulations with SimuEvol, thus obtaining alignment.
    3. Estimate site-specific amino-acid preferences with PhyloBayes using the alignment.
    4. Compate the estimation to the original site-specific amino-acid preferences 

Requirements: 

Dataset: [Bloom 2017](https://biologydirect.biomedcentral.com/articles/10.1186/s13062-016-0172-z)

SimuEvol: https://github.com/ThibaultLatrille/SimuEvol

PhyloBayes MPI: https://github.com/bayesiancook/pbmpi2

DMS_tools: https://jbloomlab.github.io/dms_tools/index.html
    
## global imports, define subdirectories and input files


In [ ]:
# GLOBAL IMPORTS
from IPython.display import display
from wand.image import Image as WImage

current_dir = "/home/thibault/SimuEvol"
protein = "lactamase"

## Plot of the initial amino-acid preferences


In [ ]:
prefs_path = "{0}/data/{1}.txt".format(current_dir, protein)
pdf_path = "{0}/data/{1}.pdf".format(current_dir, protein)

nperline = 53
log = !dms_logoplot {prefs_path} {pdf_path} --nperline {nperline}

def plot(plot_name, size='900x'):
    print("Here is the selection plot {0}".format(plot_name))
    img = WImage(filename=plot_name, resolution=480)
    img.transform(resize=size)
    display(img)
    
plot(pdf_path)

## Forward simulation in C++

Producing a alignment in .ali format from a preferences file and a newick tree.

In [ ]:
simu_evol_path = "{0}/cmake-build-debug/SimuEvol".format(current_dir)
ali_path = "{0}/data/{1}.ali".format(current_dir, protein)

!{simu_evol_path} --protein=lactamase > {ali_path}
!cat {ali_path}

## Preferences estimation with PhyloBayes

In [ ]:
nbr_cpu = 8
points = 500
newick_path = "{0}/data/{1}.newick".format(current_dir, protein)
pb_data_path = "{0}/data/pb_{1}".format(current_dir, protein)

!mpirun -n {nbr_cpu} pb_mpi -f -s -x 1 {points} -mutsel -dp -d {ali_path} -T {newick_path} {pb_data_path}

In [ ]:
!readpb_mpi -x 100 -om {pb_data_path}
!readpb_mpi -x 100 -ss {pb_data_path}

## Plot estimated amino-acid preferences

In [ ]:
pb_siteprofiles = '{0}.siteprofiles'.format(pb_data_path)
pb_prefs = '{0}_prefs.txt'.format(pb_data_path)

with open(pb_siteprofiles, 'r') as r:
    with open(pb_prefs, 'w') as w:
        r.readline()
        w.write("# POSITION WT SITE_ENTROPY PI_A PI_C PI_D PI_E PI_F PI_G PI_H PI_I PI_K PI_L PI_M PI_N PI_P PI_Q PI_R PI_S PI_T PI_V PI_W PI_Y\n")
        for line in r:
            line_split = line.replace('\n', '').split('\t')
            w.write(line_split[0] + " A 1.0 " + " ".join(line_split[1:]) + "\n")

prefsplot = '{0}_prefs.pdf'.format(pb_data_path)

log = !dms_logoplot {pb_prefs} {prefsplot} --nperline {nperline}

plot(prefsplot)

In [ ]:
diff_prefs = "{0}/data/diff_{1}".format(current_dir, protein)

log = !dms_merge {diff_prefs} sum {prefs_path} --minus {pb_prefs}

diff_plot = '{0}.pdf'.format(diff_prefs)
log = !dms_logoplot {diff_prefs} {diff_plot} --nperline {nperline}

plot(diff_plot)

corr_plot = '{0}_corr'.format(diff_prefs)
log = !dms_correlate {prefs_path} {pb_prefs} {corr_plot} --name1 "Initial" --name2 "Estimated"

plot('{0}.pdf'.format(corr_plot), size='500x')
!cat {corr_plot}.txt